# Preprocesamiento

##### General
    - Me quedo solo con los datos de test final

##### Postulantes Educacion:
    - Borro duplicados
    - One Hot Encoder para el nombre de los estudios
    - Label Encoder para el estado
    
##### Postulantes Genero y Edad
    - Borro duplicados
    - Agrego la edad
    - Reemplazo los NaN con la mediana
    - Separo edad en rangos
    
##### Vistas:
    - Borro duplicados
    
##### Postulaciones:
    - Borro duplicados
    
##### Avisos Online:
    - Borro duplicados
    
##### Avisos Detalle:
    - Borro duplicados
    - One Hot Encoder para el area de trabajo
    

In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split

In [2]:
def oneHotEncoding(df, columna):
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Load CSV

In [3]:
test_final = pd.read_csv('data/test_final_100k.csv')
test_final = test_final.drop(columns='id')

In [4]:
# idpostulantes
postulantes_educacion = pd.read_csv('data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.reset_index(drop=True)

In [5]:
# idpostulantes
postulantes_genero_y_edad = pd.read_csv('data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [6]:
# idpostulantes idaviso
vistas = pd.read_csv('data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index(drop=True)
vistas = vistas.rename(columns={'idAviso':'idaviso'})

In [7]:
# idpostulantes idaviso
postulaciones = pd.read_csv('data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index(drop=True)

In [8]:
# idaviso
avisos_online = pd.read_csv('data/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv'))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')
avisos_online = avisos_online.reset_index(drop=True)

In [9]:
# idaviso
avisos_detalle = pd.read_csv('data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index(drop=True)

# 0. General

In [ ]:
# gente_solo_test
gente = test_final['idpostulante'].drop_duplicates()
postulantes_educacion = postulantes_educacion[postulantes_educacion['idpostulante'].isin(gente)]
postulantes_genero_y_edad = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].isin(gente)]
vistas = vistas[vistas['idpostulante'].isin(gente)]
postulaciones = postulaciones[postulaciones['idpostulante'].isin(gente)]

In [ ]:
# avisos_solo_test
avisos = test_final['idaviso'].drop_duplicates()
vistas = vistas[vistas['idAviso'].isin(avisos)]
postulaciones = postulaciones[postulaciones['idaviso'].isin(avisos)]
avisos_online = avisos_online[avisos_online['idaviso'].isin(avisos)]
avisos_detalle = avisos_detalle[avisos_detalle['idaviso'].isin(avisos)]

In [10]:
# gente pge,pe y appendeo gente test
gente = postulantes_genero_y_edad.merge(postulantes_educacion,how='outer',on='idpostulante',indicator=True)
gente = gente[gente['_merge'] == 'both']
gente = gente.drop(columns='_merge')
gente = gente['idpostulante'].drop_duplicates()
gente = gente.append(test_final['idpostulante'])
gente = gente.drop_duplicates()
postulantes_educacion = postulantes_educacion[postulantes_educacion['idpostulante'].isin(gente)]
postulantes_genero_y_edad = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].isin(gente)]
vistas = vistas[vistas['idpostulante'].isin(gente)]
postulaciones = postulaciones[postulaciones['idpostulante'].isin(gente)]

In [10]:
# avisos solo detalles
avisos = avisos_detalle['idaviso'].drop_duplicates()
vistas = vistas[vistas['idaviso'].isin(avisos)]
postulaciones = postulaciones[postulaciones['idaviso'].isin(avisos)]
avisos_online = avisos_online[avisos_online['idaviso'].isin(avisos)]
avisos_detalle = avisos_detalle[avisos_detalle['idaviso'].isin(avisos)]

In [11]:
# tasa de postulaciones segun aviso
postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'])
postulaciones['fechapostulacion'] = postulaciones['fechapostulacion'].dt.date
vistas['timestamp'] = pd.to_datetime(vistas['timestamp'])
vistas['timestamp'] = vistas['timestamp'].dt.date

postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idaviso').agg({'idpostulante':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idaviso').agg({'idpostulante':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idaviso = postulaciones_validas.merge(vistas_validas,on='idaviso')
tasa_idaviso['tasa'] = tasa_idaviso['idpostulante_x'] / tasa_idaviso['idpostulante_y']
tasa_idaviso = tasa_idaviso.drop(columns=['idpostulante_x','idpostulante_y'])

In [18]:
# tasa de postulaciones segun postulante
postulaciones_validas = postulaciones[postulaciones['fechapostulacion'].isin(vistas['timestamp'].drop_duplicates())]
vistas_validas = vistas[vistas['timestamp'].isin(postulaciones_validas['fechapostulacion'].drop_duplicates())]

postulaciones_validas = postulaciones_validas.groupby('idpostulante').agg({'idaviso':'count'})
postulaciones_validas = postulaciones_validas.reset_index()
vistas_validas = vistas_validas.groupby('idpostulante').agg({'idaviso':'count'})
vistas_validas = vistas_validas.reset_index()

tasa_idpostulante = postulaciones_validas.merge(vistas_validas,on='idpostulante')
tasa_idpostulante['tasa'] = tasa_idpostulante['idaviso_x'] / tasa_idpostulante['idaviso_y']
tasa_idpostulante = tasa_idpostulante.drop(columns=['idaviso_x','idaviso_y'])

In [21]:
tasa_idpostulante.head()

,idpostulante,tasa
0,0z5Dmrd,0.400000
1,0z5JW1r,0.214286
2,0z5VvGv,1.000000
3,0zB01pE,0.400000
4,0zB026d,0.300000


In [14]:
tasa_idaviso.head()

,idaviso,tasa
0,12543760,0.481108
1,12812680,0.755556
2,17903700,0.070826
3,1000139175,0.393586
4,1000365835,0.247863


# 1. Postulantes Educacion

In [ ]:
postulantes_educacion.head()

In [ ]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')
postulantes_educacion = postulantes_educacion.reset_index(drop=True)

In [ ]:
dic = {'Graduado': 3,'En Curso': 2,'Abandonado': 1}
postulantes_educacion['estado'] = postulantes_educacion['estado'].map(dic)
postulantes_educacion = postulantes_educacion.groupby(['idpostulante','nombre']).max().unstack(fill_value=0)
postulantes_educacion.columns = postulantes_educacion.columns.droplevel()
postulantes_educacion = postulantes_educacion.reset_index()
postulantes_educacion = postulantes_educacion.rename_axis('',axis='columns')

In [ ]:
postulantes_educacion.head()

In [ ]:
postulantes_educacion.to_csv('data_preprocesada/fiuba_1_postulantes_educacion.csv')
del postulantes_educacion

# 2. Postulantes Genero y Edad

In [22]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [23]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante',keep='last')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [24]:
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year
postulantes_genero_y_edad.loc[postulantes_genero_y_edad['edad'].isna(),'edad'] = postulantes_genero_y_edad['edad'].median()
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype('int16')
postulantes_genero_y_edad = postulantes_genero_y_edad.drop(columns='fechanacimiento')
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index(drop=True)

In [26]:
postulantes_genero_y_edad = postulantes_genero_y_edad.merge(tasa_idpostulante,how='left',on='idpostulante')

In [29]:
postulantes_genero_y_edad = postulantes_genero_y_edad.fillna(0)

In [30]:
postulantes_genero_y_edad.head()

,idpostulante,sexo,edad,tasa
0,eo2p,MASC,37,0.0
1,1d2B,MASC,42,0.0
2,EBO0,FEM,45,0.0
3,a6MKW,MASC,44,0.0
4,6MWd4,MASC,44,0.0


In [31]:
postulantes_genero_y_edad.to_csv('data_preprocesada/fiuba_2_postulantes_genero_y_edad.csv')
del postulantes_genero_y_edad

# 3. Vistas

In [ ]:
vistas.head()

In [ ]:
vistas = vistas.drop_duplicates(keep='last')
vistas = vistas.reset_index(drop=True)

In [ ]:
vistas = vistas.drop(columns='timestamp')

In [ ]:
# Cantidad de veces que se visito un aviso
vistas['visto'] = 1
vistas = vistas.groupby(['idaviso','idpostulante']).max()
#vistas = vistas.drop_duplicates(keep='last')
vistas = vistas.reset_index()

In [ ]:
vistas.head()

In [ ]:
#vistas = vistas.append(postulaciones)
#vistas = vistas.fillna(1)
#vistas['visto'] = vistas['visto'].astype('int8')
#vistas = vistas.drop_duplicates()

In [ ]:
vistas.to_csv('data_preprocesada/fiuba_3_vistas.csv')
del vistas

# 4. Postulaciones

In [ ]:
postulaciones.head()

In [ ]:
postulaciones = postulaciones.drop(columns='fechapostulacion')

In [ ]:
postulaciones = postulaciones.drop_duplicates()
postulaciones = postulaciones.reset_index(drop=True)

In [ ]:
postulaciones.head()

In [ ]:
postulaciones.to_csv('data_preprocesada/fiuba_4_postulaciones.csv')
#del postulaciones

# 5. Avisos Online

In [ ]:
avisos_online.head()

In [ ]:
avisos_online = avisos_online.drop_duplicates(keep='last')
avisos_online = avisos_online.reset_index(drop=True)

In [ ]:
avisos_online['online'] = 1

In [ ]:
avisos_online.head()

In [ ]:
avisos_online.to_csv('data_preprocesada/fiuba_5_avisos_online.csv')
del avisos_online

# 6. Avisos Detalle

In [ ]:
avisos_detalle.head()

In [ ]:
avisos_detalle = avisos_detalle.drop(columns=['idpais','titulo','descripcion','nombre_zona','ciudad','mapacalle','tipo_de_trabajo','nivel_laboral','denominacion_empresa'])

In [ ]:
avisos_detalle = avisos_detalle.drop_duplicates('idaviso',keep='last')
avisos_detalle = avisos_detalle.reset_index(drop=True)

In [ ]:
replacements = {
   'nombre_area': {
      'Ingeniería Geológica|Ingeniería Metalúrgica|Ingeniería  Metalurgica|Ingeniería  Eléctrica y Electrónica|Ingeniería  Automotriz|Ingeniería de Producto|Ingeniería de Procesos|Ingeniería de Ventas|Ingeniería Oficina Técnica / Proyecto|Ingeniería en Alimentos|Ingeniería Agrónoma|Ingeniería Química|Ingeniería  Industrial|Ingeniería Electromecánica|Ingeniería  Mecánica|Otras Ingenierias': 'Ingenieria',      
      'Corporate Finance / Banca Inversión|Evaluación Económica|Organización y Métodos|Cuentas Corrientes|Cadetería|Contabilidad|Impuestos|Auditoría|Control de Gestión|Consultoria': 'Administración',#cuentasc podria ir en ventas
      'Diseño Multimedia|Diseño 3D|Diseño Textil e Indumentaria|Diseño Web|Diseño de Interiores / Decoración|Diseño Industrial|Diseño Gráfico': 'Diseño y Urbanismo',
      'Periodismo|Comunicaciones Internas|Relaciones Institucionales/Publicas|Responsabilidad Social': 'Comunicacion',
      'Auditoría de Seguros|Venta de Seguros|Tecnico de Seguros|Administración de Seguros|Administracion de Seguros': 'Seguros',
      'Quimica|Bioquimica|Bioquímica': 'Química',
      'Selección|Capacitación|Compensación y Planilla|Administración de Personal': 'Recursos Humanos', #administracion de personal dudo
      'Exploración Minera y Petroquimica|Ingeniería en Petróleo y Petroquímica|Salud Minera|Seguridad Industrial|Medio Ambiente|Instrumentación Minera|Ingeniería en Minas|Mineria/Petroleo/Gas': 'Minería',
      'Traduccion': 'Idiomas',
      'Mantenimiento|Calidad|Programación de producción': 'Producción',
      'Inversiones / Proyectos de Inversión|Finanzas Internacionales|Análisis de Riesgos|Tesorería|Créditos y Cobranzas|Facturación|Planeamiento económico-financiero': 'Finanzas',
      'Telefonista|Call Center|Telemarketing|Caja': 'Atención al Cliente', #no se si caja va aca
      'Almacén / Depósito / Expedición|Compras|Distribución|Abastecimiento' : 'Logística',
      'Legal|Asesoría Legal Internacional': 'Legales',
      'Sistemas|Telecomunicaciones|Tecnologia/Sistemas de producción|Oficios Informática|Tecnologías de la Información|Data Warehousing|Redes|Data Entry|Administración de Base de Datos|Seguridad Informática|Soporte Técnico|Testing / QA / QC|Internet|Infraestructura|Análisis Funcional|Liderazgo de Proyecto|Programación' : 'Tecnologia/Sistemas',
      'Oficios y Profesiones|Producción y Limpieza|Estética y Cuidado Personal|Mantenimiento y Limpieza|Seguridad|Servicios|Promotoras/es|Jóvenes Profesionales': 'Oficios',#aca tambien
      'Recepcionista|Asistente': 'Secretaria',
      'Dirección de Obra|Arquitectura|Ingeniería Civil|Urbanismo|Operaciones|Diseño y Construcción|Topografía' : 'Construcción',
      'Compras Internacionales/Importación|Ventas Internacionales/Exportación|Consultorías Comercio Exterior|Apoderado Aduanal|Logística Internacionales/Importación': 'Comercio Exterior',
      'Business Intelligence|E-commerce|Community Management|Producto|Media Planning|Mercadotecnia Internacional': 'Marketing',
      'Camareros|Hotelería|Turismo': 'Gastronomia',
      'Seguridad Industrial|Oficios e Higiene' : 'Seguridad e Higiene',
      'Comercial|Ventas económico-financiero|Planeamiento Comercial|Planeamiento|Desarrollo de Negocios|Negocios Internacionales|Ventas comercial' : 'Ventas',
      'Laboratorio|Farmacia industrial|Farmacia comercial|Medicina|Otras Especialidades médicas|Diagnóstico por imágenes|Esterilización|Maternidad|Farmacéutica|Veterinaria|Odontología|Farmacia hospitalaria|Farmacia Industrial|Farmacia Comercial|Medicina Laboral|Instrumentación|Auditoría Médica|Emergentología|Kinesiología / terapia ocupacional|Prácticas cardiológicas|Otras áreas técnicas en salud|Fonoaudiología|Clínica Médica|Cuidados Intensivos Cardiologicos Adultos|Psicología|Instrumentación quirúrgica|Administración Médica' : 'Salud',
      'Gerencia / Dirección General|Dirección': 'Gerencia',
      'Pasantía / Trainee|Prácticas Profesionales': 'Pasantía',
      'Tecnologia / Tecnologia/Sistemas': 'Tecnologia/Sistemas',
      'Diseño y Construcción':'Construcción',
      'Secretaria de Tráfico': 'Logística',
      'Ventas comercial':'Ventas',
      'Salud Laboral|Salud quirúrgica':'Salud',
      'Educación/ Docentes|Educación especial|Trabajo social|Trabajo Social|Investigación y Desarrollo|Bienestar Estudiantil': 'Educación',
      'Arte y Cultura|Comunicaciones Externas|Independientes|Transporte|Creatividad|Multimedia|Back Office': 'Otros' #transporte entraria tanto en logistica como en oficios y creatividad en marketing y diseño
       #Multimedia en marketing y diseño. Back Office puede entrar en muchos
   }
}

avisos_detalle.replace(replacements, regex=True, inplace=True)

In [ ]:
avisos_detalle.loc[avisos_detalle['nombre_area'] =='Enfermería (ver Enfermería)', "nombre_area"] = "Salud"

In [ ]:
avisos_detalle = oneHotEncoding(avisos_detalle,'nombre_area')

In [ ]:
# agrego tasa
avisos_detalle = avisos_detalle.merge(tasa,how='left',on='idaviso')
avisos_detalle = avisos_detalle.fillna(0)

In [ ]:
avisos_detalle.head()

In [ ]:
avisos_detalle.to_csv('data_preprocesada/fiuba_6_avisos_detalle.csv')
del avisos_detalle